In [39]:
class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol,
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)

    def __repr__(self):
        return str(self.as_dict())

    def __eq__(self, other):
        return isinstance(other, self.__class__) and self.as_dict() == other.as_dict()


class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

    def as_dict(self):
        return dict(
            symbol=self.symbol,
            timestamp=self.timestamp,
            order=self.order,
            price=self.price,
            volume=self.volume,
            commission=self.commission)

    def __repr__(self):
        return str(self.as_dict())

    def __eq__(self, other):
        return isinstance(other, self.__class__) and self.as_dict() == other.as_dict()



In [27]:
from datetime import date, datetime
from decimal import Decimal
import json
import marshmallow

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22),
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22),
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],

    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [28]:
class StockShema(marshmallow.Schema):
    symbol = marshmallow.fields.Str()
    date = marshmallow.fields.Date()
    open = marshmallow.fields.Decimal(as_string=True)
    high = marshmallow.fields.Decimal(as_string=True)
    low = marshmallow.fields.Decimal(as_string=True)
    close = marshmallow.fields.Decimal(as_string=True)
    volume = marshmallow.fields.Integer()

    @marshmallow.post_load
    def create_stock(self, data, **kwargs):
        data['open_'] = data.pop('open')
        return Stock(**data)


class TradeSchema(marshmallow.Schema):
    symbol = marshmallow.fields.Str()
    timestamp = marshmallow.fields.DateTime()
    order = marshmallow.fields.Str()
    price = marshmallow.fields.Decimal(as_string=True)
    volume = marshmallow.fields.Decimal(as_string=True)
    commission = marshmallow.fields.Integer()

    @marshmallow.post_load
    def create_trade(self, data, **kwargs):
        return Trade(**data)


class ActivitySchema(marshmallow.Schema):
    quotes = marshmallow.fields.Nested(StockShema, many=True)
    trades = marshmallow.fields.Nested(TradeSchema, many=True)

In [29]:
encoded = ActivitySchema().dumps(activity)
print(encoded)

{"quotes": [{"symbol": "TSLA", "date": "2018-11-22", "open": "338.19", "high": "338.64", "low": "337.60", "close": "338.19", "volume": 365607}, {"symbol": "AAPL", "date": "2018-11-22", "open": "176.66", "high": "177.25", "low": "176.64", "close": "176.78", "volume": 3699184}, {"symbol": "MSFT", "date": "2018-11-22", "open": "103.25", "high": "103.48", "low": "103.07", "close": "103.11", "volume": 4493689}], "trades": [{"symbol": "TSLA", "timestamp": "2018-11-22T10:05:12", "order": "buy", "price": "338.25", "volume": "100", "commission": 9}, {"symbol": "AAPL", "timestamp": "2018-11-22T10:30:05", "order": "sell", "price": "177.01", "volume": "20", "commission": 9}]}


In [30]:
from pprint import pprint

decoded = ActivitySchema().loads(encoded)
pprint(decoded)

{'quotes': [{'symbol': 'TSLA', 'date': datetime.date(2018, 11, 22), 'open': Decimal('338.19'), 'high': Decimal('338.64'), 'low': Decimal('337.60'), 'close': Decimal('338.19'), 'volume': 365607},
            {'symbol': 'AAPL', 'date': datetime.date(2018, 11, 22), 'open': Decimal('176.66'), 'high': Decimal('177.25'), 'low': Decimal('176.64'), 'close': Decimal('176.78'), 'volume': 3699184},
            {'symbol': 'MSFT', 'date': datetime.date(2018, 11, 22), 'open': Decimal('103.25'), 'high': Decimal('103.48'), 'low': Decimal('103.07'), 'close': Decimal('103.11'), 'volume': 4493689}],
 'trades': [{'symbol': 'TSLA', 'timestamp': datetime.datetime(2018, 11, 22, 10, 5, 12), 'order': 'buy', 'price': Decimal('338.25'), 'volume': Decimal('100'), 'commission': 9},
            {'symbol': 'AAPL', 'timestamp': datetime.datetime(2018, 11, 22, 10, 30, 5), 'order': 'sell', 'price': Decimal('177.01'), 'volume': Decimal('20'), 'commission': 9}]}


In [31]:
class CustomEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Stock) or isinstance(o, Trade):
            res = o.as_dict()
            res['Object'] = o.__class__.__name__
            return res
        elif isinstance(o, datetime):
            return o.isoformat()
        elif isinstance(o, date):
            return o.strftime('%Y-%m-%d')
        elif isinstance(o, Decimal):
            return str(o)
        else:
            super().default(o)

In [36]:
serialized = json.dumps(activity, cls=CustomEncoder)
serialized

'{"quotes": [{"symbol": "TSLA", "date": "2018-11-22", "open": "338.19", "high": "338.64", "low": "337.60", "close": "338.19", "volume": 365607, "Object": "Stock"}, {"symbol": "AAPL", "date": "2018-11-22", "open": "176.66", "high": "177.25", "low": "176.64", "close": "176.78", "volume": 3699184, "Object": "Stock"}, {"symbol": "MSFT", "date": "2018-11-22", "open": "103.25", "high": "103.48", "low": "103.07", "close": "103.11", "volume": 4493689, "Object": "Stock"}], "trades": [{"symbol": "TSLA", "timestamp": "2018-11-22T10:05:12", "order": "buy", "price": "338.25", "volume": 100, "commission": "9.99", "Object": "Trade"}, {"symbol": "AAPL", "timestamp": "2018-11-22T10:30:05", "order": "sell", "price": "177.01", "volume": 20, "commission": "9.99", "Object": "Trade"}]}'

In [37]:
def decode_stock(d):
    s = Stock(d['symbol'],
              datetime.strptime(d['date'],
                                '%Y-%m-%d', ).date(),
              Decimal(d['open']),
              Decimal(d['high']),
              Decimal(d['low']),
              Decimal(d['close']),
              int(d['volume']))
    return s


def decode_trade(d):
    s = Trade(d['symbol'],
              datetime.strptime(d['timestamp'], '%Y-%m-%dT%H:%M:%S'),
              d['order'],
              Decimal(d['price']),
              int(d['volume']),
              Decimal(d['commission']))
    return s


def decode_activity(d):
    clas_type = d.get('Object')
    if clas_type == 'Stock':
        return decode_stock(d)
    elif clas_type == 'Trade':
        return decode_trade(d)
    return d


class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        obj = json.loads(arg)
        return self.parse_activity(obj)

    def parse_activity(self, obj):
        if isinstance(obj, dict):
            obj = decode_activity(obj)
            if isinstance(obj, dict):
                for k, v in obj.items():
                    obj[k] = self.parse_activity(v)
        elif isinstance(obj, list):
            for indx, item in enumerate(obj):
                obj[indx] = self.parse_activity(item)
        return obj


In [41]:
json.loads(serialized, cls=CustomDecoder)

{'quotes': [{'symbol': 'TSLA', 'date': datetime.date(2018, 11, 22), 'open': Decimal('338.19'), 'high': Decimal('338.64'), 'low': Decimal('337.60'), 'close': Decimal('338.19'), 'volume': 365607},
  {'symbol': 'AAPL', 'date': datetime.date(2018, 11, 22), 'open': Decimal('176.66'), 'high': Decimal('177.25'), 'low': Decimal('176.64'), 'close': Decimal('176.78'), 'volume': 3699184},
  {'symbol': 'MSFT', 'date': datetime.date(2018, 11, 22), 'open': Decimal('103.25'), 'high': Decimal('103.48'), 'low': Decimal('103.07'), 'close': Decimal('103.11'), 'volume': 4493689}],
 'trades': [{'symbol': 'TSLA', 'timestamp': datetime.datetime(2018, 11, 22, 10, 5, 12), 'order': 'buy', 'price': Decimal('338.25'), 'volume': 100, 'commission': Decimal('9.99')},
  {'symbol': 'AAPL', 'timestamp': datetime.datetime(2018, 11, 22, 10, 30, 5), 'order': 'sell', 'price': Decimal('177.01'), 'volume': 20, 'commission': Decimal('9.99')}]}